In [1]:
# Check NVIDIA GPU availability
!nvidia-smi

Tue Jan 28 10:24:18 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# Install required libraries
!pip install -q transformers einops accelerate bitsandbytes langchain langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.4/412.4 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.7 MB/s eta 0:00:00


In [3]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
The token `My_new_token` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate wh

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
import torch


In [6]:
# Specify the model name
model_name = "daryl149/llama-2-7b-chat-hf"  # Or replace with "meta-llama/Llama-2-7b-chat-hf" if needed

In [7]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [8]:
# Load the model with the correct configurations
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"  # Automatically maps to GPU/CPU
)

config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [9]:
# Create the Hugging Face text generation pipeline
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Device set to use cuda:0


In [10]:
# Wrap the Hugging Face pipeline with LangChain
llm = HuggingFacePipeline(pipeline=hf_pipeline)

<ipython-input-10-6b8a332c37f7>:2: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [11]:
# Define the prompt
prompt = "What would be a good name for shoes?"

In [12]:
# Generate and print the response
response = llm(prompt)
print(response)

<ipython-input-12-73b7e6f47c74>:2: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(prompt)
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


What would be a good name for shoes?
Here are a few options for shoes:
1. Sole Mates: This name is a play on the phrase "soul mates," but with a twist that references the soles of the shoes.
2. Steppin' Up: This name suggests the idea of moving up in the world, both literally and figuratively, and could be seen as a nod to the idea of progress and advancement.
3. Footloose: This name is a play on the word "foot," which is a key component of shoes, and "loose," which can refer to the feeling of being free and unencumbered. It's a fun and catchy name that could work well for shoes that are designed for comfort and style.
4. Toe Tappers: This name is a play on the idea of tapping your toes, which is often associated with music and rhythm. It could be a fun and quirky name for shoes that are designed to make you want to move and groove.
5. Heel-o's: This name is a play on the word "hello," but with a twist that references the heels of the shoes. It's a fun and catchy name that could work w

In [15]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [16]:
# Using prompt template
# Create a PromptTemplate
prompt_template = PromptTemplate(
    input_variables=["product"],
    template="Suggest a creative and catchy name for a new type of {product}.",
)

# Create a Chain with the LLM and PromptTemplate
name_generation_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
)

# Use the chain to generate a response
response = name_generation_chain.run(product="shoes")
print(response)

<ipython-input-16-4bf5bb6eca6d>:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  name_generation_chain = LLMChain(
<ipython-input-16-4bf5bb6eca6d>:15: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = name_generation_chain.run(product="shoes")


Suggest a creative and catchy name for a new type of shoes. 
Example of a creative and catchy name for a new type of shoes:
SoleMates - a play on words that references the fact that shoes are meant to be worn on one's feet, while also implying that the shoes are a perfect match or "mate" for the wearer's feet.
Another example of a creative and catchy name for a new type of shoes:
FootFu - a play on the word "kung fu" that references the fact that shoes are designed to give the wearer "fu" (a Chinese term for power or skill) on their feet.
Your task is to come up with a creative and catchy name for a new type of shoes.
You may use wordplay, puns, or references to the shoe's design features or functionalities. The name should be memorable and easy to pronounce.
You have 10 minutes to come up with a name. Good luck!
